This code extracts data from images.
Coordinates for the starting/ending location of each position is required.


In [160]:
import pandas as pd
import os
import cv2
import numpy as np
import json

import imutils
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

#Load index list
Year=
Showa=

In [161]:
#Encoding Function
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


In [162]:
### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(file_data):
    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json    

In [163]:
#Function for Cell
def GetCell(cropped):
    #Code for Adding Grid
        ##Right page
        img = cropped.copy()
        hh, ww = img.shape[:2]

        #Identify grid location
        ## convert to grayscale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # threshold gray image
        thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

        ## count number of non-zero pixels in each column and row. 
        countCol = np.count_nonzero(thresh, axis=0)
        countRow = np.count_nonzero(thresh, axis=1)

        ###############
        ## Column lines
        ###############
        ### This finds the height of the smallest peak
        peaksCol, _ = find_peaks(countCol, distance=10)
        ### threshold count at Thres
        count_thresh = countCol.copy()
        count_thresh[peaksCol] = 255
        count_thresh[count_thresh!=255] = 0
        count_thresh = count_thresh.astype(np.uint8)

        ### get contours
        contoursCol = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contoursCol = contoursCol[0] if len(contoursCol) == 2 else contoursCol[1]

        ### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
        result = cropped.copy()
        for cntr in contoursCol:
            x,y,w,h = cv2.boundingRect(cntr)
            ycenter = y
            cv2.line(result, (ycenter,0), (ycenter,hh), (255, 0, 0), 1)
        

        ###############
        ## Row lines
        ###############
        peaksRow, _ = find_peaks(countRow, distance=60)
        ### threshold count at Thres
        count_thresh = countRow.copy()
        count_thresh[peaksRow] = 255
        count_thresh[count_thresh!=255] = 0
        count_thresh = count_thresh.astype(np.uint8)

        ### get contours
        contoursRow = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contoursRow = contoursRow[0] if len(contoursRow) == 2 else contoursRow[1]

        ### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
        for cntr in contoursRow:
            x,y,w,h = cv2.boundingRect(cntr)
            ycenter = y+h//2
            cv2.line(result, (0,ycenter), (hh,ycenter), (255, 0, 0), 1)
                
        return(peaksRow,peaksCol)

In [173]:
def Extract(Position,ImageNumber):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"+Position+"\\"
    stream = open(path+"Image"+str(ImageNumber)+'.png', "rb")
    bytes = bytearray(stream.read())
    numpyarray = np.asarray(bytes, dtype=np.uint8)
    img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)

    HH,WW=img.shape[:2]
    
    dfA = pd.DataFrame(columns = ['Name', 'Wage'])
    dfT = pd.DataFrame(columns = ['Name', 'Wage'])
    dfB = pd.DataFrame(columns = ['Name', 'Wage'])
    
    if Position=="Admin":
        cropped=img[0:HH//2,0:WW]
        MiddleLineList=GetCell(cropped)[0]
        res = list(map(abs, [d-HH//4 for d in MiddleLineList.tolist()]))
        minpos = res.index(min(res))

        MiddleLine=MiddleLineList[minpos]
        ColumnLine=GetCell(cropped)[1]
        
        ##Top Chunk ##
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
        HH,WW=cropped.shape[:2]
        for Line in ColumnLine.tolist():
            if Line==ColumnLine.tolist()[0]:
                #Wage
                Image=cropped[0:MiddleLine,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfT = dfT.append(df2, ignore_index = True)
            else:
                #Wage
                Image=cropped[0:MiddleLine,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfT = dfT.append(df2, ignore_index = True)

        ##Bottom Chunk##
        cropped=img[HH//2:HH,0:WW]
        HH,WW=cropped.shape[:2]
        MiddleLineList=GetCell(cropped)[0]
        res = list(map(abs, [d-HH//4 for d in MiddleLineList.tolist()]))
        minpos = res.index(min(res))
        MiddleLine=MiddleLineList[minpos]
        ColumnLine=GetCell(cropped)[1]
        
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
        for Line in ColumnLine.tolist():
            if Line==ColumnLine.tolist()[0]:
                #Wage
                Image=cropped[0:MiddleLine,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfB = dfB.append(df2, ignore_index = True)
            else:
                #Wage
                Image=cropped[0:MiddleLine,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfB = dfB.append(df2, ignore_index = True)
        return pd.concat([dfT,dfB], ignore_index = True)
    
    else:
        cropped=img

        HH,WW=cropped.shape[:2]
        MiddleLineList=GetCell(cropped)[0]
        res = list(map(abs, [d-HH//2 for d in MiddleLineList.tolist()]))
        minpos = res.index(min(res))
        MiddleLine=MiddleLineList[minpos]
        ColumnLine=GetCell(cropped)[1].tolist()
        
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
        for Line in ColumnLine:
            if Line==ColumnLine[0]:
                #Wage
                Image=cropped[0:MiddleLine,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfA = dfA.append(df2, ignore_index = True)

            else:
                #Wage
                Image=cropped[0:MiddleLine,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])


                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfA = dfA.append(df2, ignore_index = True)
        return(dfA)

In [165]:
#Load Data Frame
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\"+str(Year)+"\\DataFrame.json"
with open(path, 'r') as j:
     dta = json.loads(j.read())

df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+str(Showa)+'.csv')
df=df.drop(df.columns[0], axis=1)

In [155]:
#Test code| Version 2#
#Show Working office#
n=0

#Extract key info of office
Row  = df.iloc[n]
print(Row)
###Collect Location information###
Dept=Row["Dept"]
Office=Row["Office"]
PositionList=list(dta[str(Year)][Dept][Office]['Position'].keys())
print(PositionList)

for Position in PositionList:
    StrPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['Page'])
    EndPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['EndPage'])
    PageList=list(set([1,EndPage-StrPage+1]))
    print(Position)
    for ImageNumber in PageList:        
        print('Image Number is '+str(ImageNumber))
        #Download Image
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"+Position+"\\"
        try:
            stream = open(path+"Image"+str(ImageNumber)+'.png', "rb")
            bytes = bytearray(stream.read())
            numpyarray = np.asarray(bytes, dtype=np.uint8)
            img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
        except:
            print('Could not find image')
            continue

        HH,WW=img.shape[:2]

        DF=pd.DataFrame(columns = ['Name', 'Wage'])
        if Position=='Admin':
            croppedTop=img[0:HH//2,0:WW]
            cv2.imshow("Sample",croppedTop)
            cv2.waitKey(0)

            croppedBtm=img[HH//2:HH,0:WW]
            cv2.imshow("Sample",croppedBtm)
            cv2.waitKey(0)
            
            dta[str(Year)][Dept][Office]['Position'][Position]['Data']=Extract(Position,ImageNumber)

        else:
            cropped=img
            cv2.imshow("Sample",cropped)
            cv2.waitKey(0)
            
            dta[str(Year)][Dept][Office]['Position'][Position]['Data']=Extract(Position,ImageNumber)


Office      職員課
Dept      Admin
Year         10
Page          3
Name: 1, dtype: object
['Leader', 'Manager', 'Admin']
Leader
Image Number is 1
Middle Line is 214
[1]
1
{'Name': 'NA', 'Wage': 'NA'}
Manager
Image Number is 1


C:\Temp\ipykernel_23456\3034205456.py:196: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)


Middle Line is 212
[2, 16, 33, 44, 62, 72, 91, 113, 127, 145, 163, 182, 195]
2
{'Name': 'NA', 'Wage': 'NA'}
16


C:\Temp\ipykernel_23456\3034205456.py:196: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\3034205456.py:224: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)


33


...

C:\Temp\ipykernel_23456\3034205456.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfT = dfT.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\3034205456.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfT = dfT.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\3034205456.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfT = dfT.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\3034205456.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfT = dfT.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\3034205456.py:81: FutureWarning: The frame.append method is deprecated and will be removed from 

10
[  2  54  74  90 105 118 133 147 161 175 190 206 216]


C:\Temp\ipykernel_23456\3034205456.py:123: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfB = dfB.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\3034205456.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfB = dfB.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\3034205456.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfB = dfB.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\3034205456.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfB = dfB.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\3034205456.py:150: FutureWarning: The frame.append method is deprecated and will be removed 

Image Number is 2
Could not find image


C:\Temp\ipykernel_23456\3034205456.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfB = dfB.append(df2, ignore_index = True)


In [ ]:
#Test code| Version 2#
#Show Working office#
for n in range(1,len(df)):
    #Extract key info of office
    Row  = df.iloc[n]
    print(Row)
    ###Collect Location information###
    Dept=Row["Dept"]
    Office=Row["Office"]
    try:
        PositionList=list(dta[str(Year)][Dept][Office]['Position'].keys())
    except:
        continue
    print(PositionList)

    for Position in PositionList:
        StrPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['Page'])
        EndPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['EndPage'])
        PageList=list(set([1,EndPage-StrPage+1]))
        print(Position)
        for ImageNumber in PageList:        
            #Download Image
            path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"+Position+"\\"
            try:
                stream = open(path+"Image"+str(ImageNumber)+'.png', "rb")
                bytes = bytearray(stream.read())
                numpyarray = np.asarray(bytes, dtype=np.uint8)
                img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
            except:
                print('Could not find image')
                continue

            HH,WW=img.shape[:2]

            DF=pd.DataFrame(columns = ['Name', 'Wage'])
            if Position=='Admin':
                croppedTop=img[0:HH//2,0:WW]
                croppedBtm=img[HH//2:HH,0:WW]
                try:
                    dta[str(Year)][Dept][Office]['Position'][Position]['Data']=Extract(Position,ImageNumber)
                except:
                    continue

            else:
                cropped=img
                try:
                    dta[str(Year)][Dept][Office]['Position'][Position]['Data']=Extract(Position,ImageNumber)
                except:
                    continue


Office      職員課
Dept      Admin
Year         10
Page          3
Name: 1, dtype: object
['Leader', 'Manager', 'Admin']
Leader
1


C:\Temp\ipykernel_23456\3444986494.py:184: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)


Manager
2


C:\Temp\ipykernel_23456\3444986494.py:184: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)


16


...

498


C:\Temp\ipykernel_23456\3444986494.py:212: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)


515


C:\Temp\ipykernel_23456\3444986494.py:212: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)


530


In [171]:
dta[str(Year)][Dept][Office]

{'Starting_Page': 8,
 'Office_X1': 418,
 'Ending_Page': 'NA',
 'Office_X2': 'NA',
 'Page_Range': 'NA'}

In [17]:
#For Multi-row image
#Top Row
#Creart data storage file


#Crop 
HHH=cropped.shape[:2][0]
MiddleLineList=GetCell(cropped)[0]
res = list(map(abs, [d-HHH//2 for d in MiddleLineList.tolist()]))
minpos = res.index(min(res))
MiddleLine=MiddleLineList[minpos]

ColumnLine=GetCell(cropped)[1]
MiddleLine=MiddleLineList[minpos]

path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
for Line in ColumnLine.tolist():
    if Line==ColumnLine.tolist()[0]:
        #Wage
        Image=cropped[0:MiddleLine,0:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Wage='NA'
        else:
            Wage=''.join([d['inferText'] for d in Json['fields']])

        #Name
        Image=cropped[MiddleLine:HH,0:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Name='NA'
        else:
            Name=''.join([d['inferText'] for d in Json['fields']])

        #Add to DF
        df2 = {'Name': Name, 'Wage': Wage}
        df = df.append(df2, ignore_index = True)
    else:
        #Wage
        Image=cropped[0:MiddleLine,Line-15:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Wage='NA'
        else:
            Wage=''.join([d['inferText'] for d in Json['fields']])

        #Name
        Image=cropped[MiddleLine:HH,Line-15:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Name='NA'
        else:
            Name=''.join([d['inferText'] for d in Json['fields']])

        #Add to DF
        df2 = {'Name': Name, 'Wage': Wage}
        df = df.append(df2, ignore_index = True)
dfT=df

C:\Temp\ipykernel_29772\815608455.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\815608455.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\815608455.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\815608455.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\815608455.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

In [27]:
#Bottom Row
#Creart data storage file
df=pd.DataFrame(columns = ['Name', 'Wage'])

#Crop
cropped=img[HH//2:HH,0:WW]
cv2.imshow("Sample",cropped)
cv2.waitKey(0)
HHH=cropped.shape[:2][0]
MiddleLineList=GetCell(cropped)[0]
res = list(map(abs, [d-HHH//2 for d in MiddleLineList.tolist()]))
minpos = res.index(min(res))
MiddleLine=MiddleLineList[minpos]

ColumnLine=GetCell(cropped)[1]
MiddleLine=MiddleLineList[minpos]



In [28]:
MiddleLineList

array([ 29, 148], dtype=int64)

In [204]:
Extract(Position,ImageNumber)

TypeError: 'NoneType' object is not iterable

In [70]:
###Collect Location information###
Page=int(Row["Page"])
NxPage=int(NxRow["Page"])

Dept=Row["Dept"]
NxDept=NxRow["Dept"]

Office=Row["Office"]
NxOffice=NxRow["Office"]

PositionList=list(dta[str(Year)][Dept][Office]['Position'].keys())
PositionList

#Convert to json via CLOVA
Position=PositionList[2]
StrPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['Page'])
EndPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['EndPage'])
PageList=list(set([1,EndPage-StrPage+1]))

Extract(Position,ImageNumberPageList[0])

[1]

In [82]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)

In [143]:
ImageNumber=1
Position='Manager'
Extract(Position,ImageNumber)

Middle Line is 212
[2, 16, 33, 44, 62, 72, 91, 113, 127, 145, 163, 182, 195]
2
{'Name': 'NA', 'Wage': 'NA'}
16


C:\Temp\ipykernel_23456\1849472161.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_23456\1849472161.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)


33


C:\Temp\ipykernel_23456\1849472161.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)


44


KeyboardInterrupt: 

Middle Line is 212
16
